In [ ]:
from __future__ import print_function

from intervaltree import IntervalTree, Interval
from collections import defaultdict
import sys
import numpy
import screed

In [ ]:
def overlap_intervals(self, other):
    splits = (self | other)
    splits.split_overlaps()
    self_int_other = IntervalTree(filter(lambda r: self.overlaps(r) and other.overlaps(r), splits))
    return self_int_other


def overlap_all(trees):
    if len(trees) in (0, 1):
        return None

    overlap = overlap_intervals(trees[0], trees[1])
    for t in trees[2:]:
        overlap = overlap_intervals(overlap, t)
    overlap.merge_overlaps(data_reducer)

    return overlap


def data_reducer(current_data, new_data):
    return current_data | new_data

In [ ]:
ref =[0,205603715 ]
gtrees = defaultdict(dict)
all_genes = set()
files = defaultdict(lambda: defaultdict(list))
ref =  defaultdict(lambda: defaultdict(list))

In [ ]:
refsizes={}
ivals={}
for record in screed.open('mircea.fa'):
        refsizes[record.name.split()[0]] = len(record.sequence)
for genome in refsizes:
        ivals[genome] = numpy.zeros(refsizes[genome])

#for f in ('id', 'mg'): 
f='idba.coords'
#f='mg'
file= open(f)
lines = file.readlines()[5:]
file.close()
for line in lines:
            x =line.split()
            s1, e1 = int(x[0]), int(x[1])
            s2, e2 =int(x[3]), int(x[4])
            ident =(x[9])
            s1 =min(s1,e1)
            e1 =max(s1,e1)
            s2 =min(s2,e2)
            e2 =max(s2,e2)
            name1=x[11]
            lgth =e2-s2 
            #print(s1,e1,s2,e2,ident,lgth,name1)
            if ident<99 or lgth <100: 
                   continue
            genome = ivals[name1]
            genome[s1 - 1:e1] = numpy.ones(e1 - s1 + 1)
            files[f][name1].append((int(s1), int(e1))) # instead of x[0], x[1]
            #files[f][x[11]].append((x[0],x[1]))
total=0.
covered=0.
uncovered=0.
for k, v in ivals.items():
          total += len(v)
          covered+=sum(v)         
uncovered =total-covered
print('All Values are:', covered, uncovered, covered+uncovered,total,covered/total, uncovered/total)

In [ ]:
gtrees = defaultdict(dict)
tt=defaultdict(dict)
ref_tree=defaultdict(dict)
all_genes = set()
for f in files:
    for g in files[f]:
        all_genes.add(g)
        gtrees[g][f] = IntervalTree(Interval(iv[0], iv[1], set([f]))
                                    for iv in files[f][g])
        ref_tree[g][f] = IntervalTree(Interval(iv[0], iv[1], set([f]))
                                    for iv in ref[f][g])
        tt[g] = IntervalTree([Interval(1, 3814139)])
print('done')

In [48]:
#uncovered intervals
covered =0.0
uncovered=0.0
total=0.0
refsizes={}
ivals={}  
uncov={}
s =0.
uncov2=0.
cov=0.
t=205603715.0
for record in screed.open('mircea.fa'):
        refsizes[record.name.split()[0]] = len(record.sequence)
for genome in refsizes:
        ivals[genome] = numpy.zeros(refsizes[genome])
        uncov[genome]= numpy.zeros(refsizes[genome])
t= IntervalTree([Interval(1, 3814139)])
    
for i in gtrees: 
   # t= IntervalTree([Interval(1, 3814139)])
    for j in gtrees[i]:                          
            for interval in sorted(gtrees[i][j]):
                    #uncov= ivals[i]
                    t.chop(interval.begin, interval.end)
                    cov+=(interval.end-interval.begin) +1         
                    #genome = ivals[i]
                    #genome[interval.begin - 1:interval.end] = numpy.ones(interval.end - interval.begin+ 1) 
                    #ivals[i][interval.begin - 1:interval.end] = numpy.ones(interval.end - interval.begin+ 1) 

    for it in t: 
        uncov2+=(it.end-it.begin)
"""       
for k, v in ivals.items(): #swapped if switched to uncov
          total += len(v)
          covered+=sum(v)
          for i in v: 
                if i ==0: 
                    uncovered+=1
uncovered =total-covered   
""" 
print('All Done', cov, uncov2)
#print ('The covered is :',covered, uncovered,covered+uncovered, total, covered/total, uncovered/total, uncov2, uncov2/total)

All Done 108754305.0 5299867.0


# Testing sample

In [49]:
gs = defaultdict(dict)
fs = defaultdict(lambda: defaultdict(list))
fs['idba']['a'].append((1,5))
fs['idba']['a'].append((10,15))
fs['idba']['a'].append((17,25))
fs['idba']['a'].append((20,25))

cov =0.
t= IntervalTree([Interval(1, 30)])
for f in fs:
    for g in fs[f]:
        gs[g][f] = IntervalTree(Interval(iv[0], iv[1], set([f]))
                                    for iv in fs[f][g])
for i in gs: 
    for j in gs[i]:                          
            for interval in sorted(gs[i][j]):
                cov+=(interval.end-interval.begin)
                t.chop(interval.begin, interval.end)
uncov=0.
for it in t: 
    uncov+=(it.end-it.begin)
print (cov,uncov) 

22.0 12.0


# Below are some stuff I tried

In [ ]:
#This works but choping has weird values
for i in ref_tree: 
        for j in ref_tree[i]:
               #print(ref_tree[i][j].begin(), ref_tree[i][j].end())   
               for a in gtrees: 
                  for b in gtrees[a]: 
                         #gtrees[i][j]=ref_tree[i][j] -gtrees[a][b]
                        #gtrees[i][j]=ref_tree[i][j] &gtrees[a][b]
                         #ref_tree[i][j].chop(gtrees[i][j].begin(), gtrees[i][j].end())   
                         print(j,ref_tree[i][j].begin(), ref_tree[i][j].end())   



In [ ]:
ref_copy=defaultdict(dict)
ref_copy =ref_tree
t = IntervalTree([Interval(1, 3814139)])
for i in ref_tree: 
     for j in ref_tree[i]:  
              for a in gtrees: 
                   for b in gtrees[a]: 
                         #print(ref_tree[i][j].begin(), ref_tree[i][j].end(), gtrees[a][b].begin(), gtrees[a][b].end())
                         #ref_tree[i][j]-=gtrees[a][b]
                         x= gtrees[a][b].begin()
                         y =gtrees[a][b].end()
                         #print (x,y)
                         #t.chop(gtrees[a][b].begin(), gtrees[a][b].end())
                         #t.chop(5,500)
                         #t.chop(x,x+700)
                         print(t)
                         #ref_tree[g][f].chop(gtrees[a][b].begin(), gtrees[a][b].end())   
                         #gtrees[a][b]-=ref_tree[i][j]
                         #print(ref_tree[i][j].begin(), ref_tree[i][j].end(), gtrees[a][b].begin(), gtrees[a][b].end())  
"""
for i in gtrees: 
        for j in gtrees[i]:                          
            for interval in sorted(gtrees[i][j]):
                    print(interval.begin, interval.end) 
            print('done with', i,j)"""
#print(t)

In [ ]:
#ivs = [(0, 1000000), (1000001,2000000), (2000001, 3000000)]
#t = IntervalTree(Interval(*iv) for iv in ivs)
#t.remove_overlap(25, 35)
#t.chop(25,35)
#sorted(t)
t = IntervalTree()
#t[1:2]=(1,2)
for f in files: 
        for g in files[f]: 
                t.addi(ref[f][g].begin(),ref[g][f].end())
                # print(t)
            
t=t&gtrees
#t-=gtrees
#print (t)
print(t)

#print(gtrees.items())
for f in files: 
        for g in files[f]:
            #t.chopi(gtrees[g][f].begin(), gtrees[g][f].end())
           #t.removei(gtrees[g][f].begin(), gtrees[g][f].end())
print (t)